In [277]:
import requests
import pandas as pd
import json
import numpy as np

# Paramètres globaux

In [70]:
RIOT_API_KEY = "RGAPI-16eab1d0-8358-45d6-853c-d5d9ce0eab21"

PATCH = "12.19.1"

CHAMPIONS_JSON_URL = "https://ddragon.leagueoflegends.com/cdn/" + PATCH + "/data/en_US/champion.json"

PLAYER_NAME = "Skeanz"

print(CHAMPIONS_JSON_URL)

https://ddragon.leagueoflegends.com/cdn/12.19.1/data/en_US/champion.json


# Liste des champions

In [71]:
from urllib.request import urlopen
data=json.loads(urlopen(CHAMPIONS_JSON_URL).read())
champions = data['data'].keys()
champions

dict_keys(['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Poppy', 'Pyke', 'Qiyana', 'Quinn', 'Rakan', 'Ramm

# Puuid du joueur

In [72]:
recup_puuid = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{PLAYER_NAME}?api_key={RIOT_API_KEY}"
reponse_puuid = requests.get(recup_puuid)
res_puuid = reponse_puuid.json()
puuid = res_puuid["puuid"]
puuid

'vzO2ozpiICei-jK6WD-59daFWSvv2dH3E9VJ_SrFPLwuRZ7LFdza9Em9Z8gcXgZEi8sYjf6yRydTNQ'

# Liste des parties du joueur

In [73]:
end = True
count = 0
total_games = []
nb_games = 100
while end:
    recup_games = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={count}&count={nb_games}&api_key={RIOT_API_KEY}"
    reponse_games = requests.get(recup_games)
    res_games = reponse_games.json()
    total_games += res_games
    count +=100
    end = (len(res_games)==100)
len(total_games)

951

# Informations sur les parties: champion et vainqueur 

L'exécution de cette cellule peut prendre du temps (2 min pour chaque chunk de 100 parties jouées par le joueur).

In [74]:
import time

D=[]
W=[]
for game in total_games:
    api_url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{game}?api_key={RIOT_API_KEY}"
    response = requests.get(api_url)
    while response.status_code==429:
        time.sleep(2)
        api_url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{game}?api_key={RIOT_API_KEY}"
        response = requests.get(api_url)
    match = response.json()
    L1,L2=[],[]
    for participant in match['info']['participants']:
        if len(L1) <5:# une fois la première équipe remplie, on remplie la deuxième
            L1.append(participant['championName'])
        else :
            L2.append(participant['championName'])
    D.append([L1,L2,(1 if match['info']['participants'][0]['win']==True else 2)])

In [75]:
D

[[['Aatrox', 'Mordekaiser', 'Kassadin', 'Aphelios', 'Amumu'],
  ['Malphite', 'Viego', 'Akali', 'Kaisa', 'Janna'],
  1],
 [['Gragas', 'Zeri', 'Veigar', 'Kayn', 'Renata'],
  ['Caitlyn', 'MissFortune', 'Swain', 'Fizz', 'Blitzcrank'],
  2],
 [['Kennen', 'Blitzcrank', 'Twitch', 'Yuumi', 'Zoe'],
  ['Gnar', 'Belveth', 'Anivia', 'Lucian', 'Nami'],
  2],
 [['Fiora', 'LeeSin', 'Kassadin', 'Tristana', 'Amumu'],
  ['Sejuani', 'Belveth', 'Akali', 'Kaisa', 'Nautilus'],
  1],
 [['Garen', 'Taliyah', 'Sylas', 'Tristana', 'Pyke'],
  ['Aatrox', 'Viego', 'Katarina', 'Sivir', 'Rakan'],
  1],
 [['Maokai', 'Graves', 'Syndra', 'Varus', 'Soraka'],
  ['Qiyana', 'Nidalee', 'Ornn', 'Xayah', 'Janna'],
  2],
 [['Riven', 'Rammus', 'Syndra', 'Lucian', 'Nami'],
  ['Fiora', 'Graves', 'Ryze', 'Samira', 'Bard'],
  2],
 [['Shen', 'Nocturne', 'Nasus', 'Jinx', 'Zilean'],
  ['Twitch', 'LeeSin', 'Sylas', 'Draven', 'Blitzcrank'],
  2],
 [['Mordekaiser', 'Graves', 'Syndra', 'Ezreal', 'Yuumi'],
  ['Sion', 'Belveth', 'Veigar', 'S

# Data processing: One Hot Encoding

In [76]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit([[champ] for champ in list(champions)])
X=enc.transform([['Jax']]).toarray()
len(X[0])

161

In [287]:
x_raw=[]
y_raw=[]
for i in range(len(D)):
    P1=np.zeros(161)
    P2=np.zeros(161)
    for j in range(5):
        P1+=enc.transform([[D[i][0][j]]]).toarray()[0]
    for j in range(5):
        P2+=enc.transform([[D[i][1][j]]]).toarray()[0]
    x_raw.append(list(P1)+list(P2))
    y_raw.append(D[i][2])
X=np.array(x_raw)
y=np.array(y_raw)

In [247]:
X

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [248]:
y

array([1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1,
       1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2,
       1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1,
       1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2,
       1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1,
       2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2,
       2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2,
       1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2,
       1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1,
       1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1,
       2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1,
       1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2,

In [215]:
X

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Entrainement d'un modèle (classification par Random Forest)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  25  24  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160]


In [289]:
from sklearn.preprocessing import OrdinalEncoder
label_encoder = OrdinalEncoder()
x = [list(champions)]
x_raw_str=[]
y_raw_str=[]

for i in range(len(D)):
    P1=[]
    P2=[]
    for j in range(5):
        P1.append(D[i][0][j])
    for j in range(5):
        P2.append(D[i][1][j])
    x_raw_str.append(list(P1)+list(P2))
X2 = np.array(x_raw_str)
X3 = label_encoder.fit_transform(X2)
X3.shape

(951, 10)

In [290]:
y.shape

(951,)

In [307]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

scores=[]
N=100
for i in range(N):
    X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.2, random_state=42)
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_test,y_test))
np.mean(scores)

0.503455497382199

In [230]:
clf.predict(X_test)

array([1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2,
       1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1,
       1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1,
       1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2,
       1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1,
       1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1,
       1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2,
       1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1,
       2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1])

In [139]:
y_test

array([2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2,
       1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1,
       2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2,
       2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2,
       1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2,
       1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1,
       2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1])